# SWIFTCON 2025 - The "fiducial" simulation pipeline

This notebook takes you through an example of running a dark-matter-only simulation and creating subhalo catalogues for subsequent analysis. The software used throughout this tutorial is the following:

* **Initial conditions**: monofonIC. [Code repository](https://bitbucket.org/ohahn/monofonic/src/master/) and latest [reference paper](https://ui.adsabs.harvard.edu/abs/2020ascl.soft08024H/abstract).
* **Cosmological integration**: SWIFT. [Code repository](https://gitlab.cosma.dur.ac.uk/swift/swiftsim) and [reference paper](https://ui.adsabs.harvard.edu/abs/2024MNRAS.530.2378S/abstract).
* **Subhalo finding**: HBT-HERONS. [Code repository](https://github.com/SWIFTSIM/HBT-HERONS) and [reference paper](https://ui.adsabs.harvard.edu/abs/2025MNRAS.543.1339F/abstract).
* **Subhalo property calculation**: SOAP. [Code repository](https://github.com/SWIFTSIM/SOAP) and [reference paper](https://ui.adsabs.harvard.edu/abs/2025JOSS...10.8252M/abstract).

Created by Victor Forouhar Moreno (forouhar@strw.leidenuniv.nl) & Rob McGibbon (mcgibbon@strw.leidenuniv.nl)

## Dependencies

Several external libraries are required to compile and/or run the codes used in this tutorial, although there is considerable overlap across codes. If you are using the provided Docker image, all libraries are pre-installed and ready to go. If you are not using the provided image, you will need to install these libraries yourself.

#### monofonIC
- FFTW
- GSL
- HDF5

#### SWIFT
- HDF5 
- MPI
- FFTW
- METIS
- GSL

#### HBT-HERONS
- CMake
- HDF5
- MPI

#### SOAP
- mpi4py
- h5py built with parallel HDF5
- Standard python modules (see requirements.txt)

In [ ]:
mkdir -p ./outputs/monofonic ./outputs/SWIFT  ./outputs/HBT-HERONS ./outputs/SOAP ./software 

# Generating initial conditions

#### Clone repository

We need to download monofonIC first, for which we clone the official repository.

In [ ]:
git clone https://bitbucket.org/ohahn/monofonic.git ./software/monofonic


#### Compiling
We then compile the code to generate the executable we will be using.

In [ ]:
cd ./software/monofonic/
mkdir build/ && cd build

# Compilation options can be optionally specified at this stage.
cmake ..
make -j 4

# Go back to original directory
cd ../../..

#### Running monofonIC

The monofonIC executable takes the path to a parameter file as a runtime argument. In this parameter file, you can specify the redshift at which the ICs will be generated, the cosmological parameters and , among other things. 

We have provided a basic parameter file in `./parameter_files/monofonic/example.conf`. Note that the following mandatory parameters have been left unspecified, as we encourage you to play with their value:

* `GridRes`: number of particles per dimension. Total number of particles will be the cube of this number.
* `BoxLength`: Length of each side of the cubic box. **It should be in `Mpc/h`!** 

TIP: if you want a lot of structure that forms at high redshift at a fixed value of `GridRes`, you can can make a "smarter" box size choice by recalling the shape of the LCDM power spectrum and/or mass function :)


In [ ]:
~/tutorial/software/monofonic/build/monofonIC ./parameter_files/monofonic/example.conf

# Running simulation

[SWIFT](https://swift.strw.leidenuniv.nl/docs/index.html) is an open-source cosmological and astrophysical numerical solver designed to run efficiently on modern hardware. A comprehensive and extensive set of models for galaxy formation as well as planetary physics are provided alongside a large series of examples.

#### Clone repository

In [ ]:
git clone https://gitlab.cosma.dur.ac.uk/swift/swiftsim.git ~/tutorial/software/swiftsim/
cd ~/tutorial/software/swiftsim/

#### Compiling

Compiling SWIFT is a bit more involved than monofonIC, see the [notes here](https://swift.strw.leidenuniv.nl/docs/GettingStarted/compiling_code.html).

Luckily for our purposes, which is to run a dark-matter-only simulation to be analysed with HBT-HERONS, we only need to pass the `--enable-fof` flag. This will make the Friends-of-Friends group algorithm available within SWIFT, which is required by HBT-HERONS to find subhaloes. 

In [ ]:
./autogen.sh
./configure --enable-fof

Then compile:

In [ ]:
make -j 4
cd ~/tutorial/outputs/SWIFT

#### Running SWIFT

We will run the non-MPI version of the code (MPI version is `swift_mpi`). Aside from the path to a parameter file, we need to specify that this simulation is cosmological (`--cosmology`), particles have gravity (`--self-gravity`) and that we want to run the Friends-of-Friends algorithm (`--fof`).

In the parameter file, you will need to specify the cosmology you chose during IC generation, as well as the gravity softening length (set it to 1/25th of the mean interparticle separation, i.e. `BoxLength / GridRes / 25`)

In [ ]:
mkdir outputs/SWIFT

In [ ]:
~/tutorial/software/swiftsim/swift --cosmology --self-gravity --fof --threads=16 ~/tutorial/parameter_files/SWIFT/configuration.yml

#### Output of run

- `snap_xxxx.hdf5` - Particles and their properties
- `fof_output_xxxx.hdf5` - FOF halo catalogue (FOF IDs stored in snapshots)
- `statistics.txt` - Global properties of the simulation over time
- `timesteps.txt` - What the simulation did during it's timesteps

# Subhalo finding

HBT-HERONS uses the FOF catalogues from SWIFT to as candidates for central subhalos. Once a subhalo has been identified at any snapshot, the particles remain associated to it even if it falls into a larger halo. Using this information makes it possible to identify satellite subhalos.

Given the history-based method of HBT-HERONS, it needs to analyse a range of snapshots from the simulation. We recommend at least ~64 snapshot spaced evenly in $\log a$.

Hydro considerations
- SWIFT particle splitting
- Which particles to use as tracers
- Number of tracers for an object to be resolved
- Thermal energy of gas particles

#### Clone repository

In [ ]:
git clone https://github.com/SWIFTSIM/HBT-HERONS ~/tutorial/software/HBT-HERONS/

#### Compiling

Several compile-time options can be set using CMAKE, which include the size of internal datatypes (for memory consideration), whether the simulation is hydrodynamical or not, and whether to include the gas internal energy during unbinding. The full list of options can be seen using `ccmake`.



In [ ]:
cd ~/tutorial/software/HBT-HERONS/
mkdir build && cd build

cmake ../ -D HBT_USE_OPENMP=ON -D HBT_DM_ONLY=ON -D HBT_UNSIGNED_LONG_ID_OUTPUT=OFF
make -j 4

cd ~/tutorial/

#### Running HBT-HERONS

The HBT-HERONS executable takes the path to a parameter file as a runtime argument. In this parameter file, you specify several parameters that relate to tracking subhaloes, how unbinding is done and the path to the simulation to analyse. 

We have provided a basic parameter file in `./parameter_files/HBT-HERONS/SWIFT.conf`. Note that you will need to add a value to `MaxSnapshotIndex` that corresponds to the total number of snapshots that the simulation you want to analyse has.

You can run HBT-HERONS over MPI and with multiple threads per rank. The number of MPI ranks determines the (spatially-based) domain decomposition.

In [ ]:
export OMP_NUM_THREADS=8
mpirun --allow-run-as-root -np 1 ~/tutorial/software/HBT-HERONS/build/HBT ~/tutorial/parameter_files/HBT-HERONS/SWIFT.conf

#### Output of run

For each snapshot each HBT rank outputs two files.
- The `SubSnap_xxx.y.hdf5` files contain information about the bound subhalos identified by HBT-HERONS, including their particles. There are only a very small number of halo properties contained in these files. All information for merger trees is also contained within these files.
- The `SrcSnap_xxx.y.hdf5` files contain a list of the particles associated with each subhalo, which can also include unbound at the current snapshot. These files are only used if HBT-HERONS if restarted, and so can be deleted once halo finding has been completed.

HBT-HERONS contains "orphan" subhalos. These are subhalos which have been disrupted, but are still tracked by the most bound particle at the time the last snapshot the subhalo was resolved.

# SOAP - Calculating halo properties

SOAP is a python package that loads in SWIFT simulation data, a subhalo catalogue containing particle bound memberships and subhalo centres, and can compute **many** properties. 

There are several different types of available aperture properties, which depend on whether a spherical cut is used to select particles or not, as well as whether the boundness of particles is accounted for. It can also automatically calculate the physical size of apertures that reach a given spherical overdensity, and disable the calculation of properties if the number of particles is insufficient to get reasonable results.

There are two steps involved in running SOAP:

* **Generating particle memberships**: SOAP loads the particle bound memberships from the subhalo catalogue and reorders them in the same order as the SWIFT simulation snapshot. This will be used later to apply boundness selections to particles during the subhalo property calculation step. You can also use these memberships to generate a virtual snapshot, to facilitate the selection of particles bound to a subhalo of interest.
* **Calculating subhalo properties**: SOAP loads the bound memberships of particles and the snapshot data, computes the requested properties, and saves an HDF5 file that is unit-aware.

#### Clone and install package

In [ ]:
git clone https://github.com/SWIFTSIM/SOAP.git ~/tutorial/software/SOAP
cd ~/tutorial/software/SOAP
pip install -e .
cd ~/tutorial/

#### Running the group membership files

Should be relatively cheap, just reads in the particles for each halo, resorts them, and outputs them in a SWIFT friendly format

In [ ]:
mpirun --allow-run-as-root -np 4 python3 -u ~/tutorial/software/SOAP/SOAP/group_membership.py \
    --sim-name=DM_test \
    --snap-nr=49      \
    ~/tutorial/parameter_files/SOAP/DMO_EXAMPLE.yml

#### Calculating halo properties

Chunks sets how to split up the simulation volume. We require multiple chunks if running on multiple nodes.

In [ ]:
mpirun --allow-run-as-root -np 4 python3 -u ~/tutorial/software/SOAP/SOAP/compute_halo_properties.py \
    --sim-name=DM_test \
    --snap-nr=49 \
    --chunks=1 \
    --dmo \
    ~/tutorial/parameter_files/SOAP/DMO_EXAMPLE.yml

#### SOAP output

Membership files have the same structure as a SWIFT snapshot. There are no particle IDs, but the particles are in the same order as the original snapshot.

SOAP catalogues have a group for each halo type. All arrays have the same length (the number of subhalos), and are always in the same order.

See tomorrow's tutorial on more information on how to use SWIFT, SOAP and merger tree data to analyse the simulation.

#### Compression, virtual snapshots, documentation (not working in docker)

The files output from SOAP can be heavily compressed. For the membership files this is because most particles are not in a halo, and there are many repeated indices for the ones that are. For the halo properties we do not compute certain properties depending on the size of the input halo, so many values are zero.

The membership files have no lossy compression filters, and can be compressed with h5repack

We can create a single virtual file which links the memebership files with the snapshot files so that the datasets in both can be accessed at the same time

In [ ]:
h5repack -i outputs/SOAP/SOAP_uncompressed/membership_0049.hdf5 -o outputs/SOAP/membership_0049.hdf5 -f GZIP=4

In [ ]:
python3 software/SOAP/compression/make_virtual_snapshot.py \
    outputs/SWIFT/snap_0049.hdf5 \
    outputs/SOAP/membership_0049/membership_0049.hdf5 \
    outputs/SOAP/snapshot_with_membership_0049.hdf5

The halo catalogues have the same lossy compression filters as are available in SWIFT (some of which are custom), and so must be compressed using the following script.

In [ ]:
python3 software/SOAP/SOAP/compression/compress_soap_catalogue.py \
    SOAP_uncompressed/halo_properties_0049.hdf5 \
    halo_properties_0049.hdf5 \
    outputs/SOAP/SOAP_uncompressed/tmp

#### Generate documentation

Documentation can be generated by running the `property_table.py`. You must pass the parameter file (to determine which halo types and properties are included in the documentation) and a SWIFT snapshot (to extract the units).

In [ ]:
cd software/SOAP
python3 SOAP/property_table.py parameter_files/SOAP_config.yml ../../outputs/SWIFT/snap_0018.hdf5
cd documentation
# pdflatex SOAP.tex
cd ../../..